# CH 20

## TOC<a id='toc'></a>
* [Ch20 Notes](#ch20_notes)

### CH20 Notes <a id='ch20_notes'></a>
[toc](#toc)
###  Attribute Descriptors

* a **descriptor** is a class that implements a protocol consisting of `__get__`, `__set__` and `__delete__` methods.
    - the property class implements the full descriptor protocol
    - also used by classmethod and staticmethod decorators
* descriptors are a distinguishing feature of pyhton deployed both at the application level and in tha language infrastructure
* a descriptor class is used by declaring instances of it as class attributes of another class.
* definitions:
    - **decriptor class** - a class implementing the descriptor protocol.
    - **managed class** - the class where the descriptor instances are declared as class attributes
    - **Descriptor Instance** -  each instance of a descriptor class declared as a class attribute of the managed class.
    - **Managed Instance** - one instance of the managed class
    - **Manageg Attribute** - a public attribute in the managed class that will be handled by a descriptor instance, with values stored in storage attributes.
    - **Storage attribute** - an attribute of the managed instance that will hold the value of a *managed attribute* for that particular instance
    
* Main point: descriptor instances are class attributes. They manage data that is stored in an instance of the managed class.

* `__set__` is called when attempting to assign to *managed attribute*
    - python provides both `self` - the descriptor instance - and `instance` the managed instance to this method, as well as the value: `__set__(self, instance, value):`

* At first, you might want to store value in descriptor isntance (self) instead of in managed instance (instance) - but that is wrong
    - descriptor instance is a class attribute - might manage many instances!

COMMENT: The example he gives seems almos exaclty the same as the property factory example - still have a storage name, still write that twice when creating property/descriptor as class attribute. Still relies on python mechanism for looking at class before instance when calling attributes ??
    - this one is weird. If I have a class var shadowed by instance var, the instance var shines through. So what is required for class attribute to override? can't be isinstance property, because descriptor does it too.

In [1]:
class MyClass:
    var = 4
    
    def __init__(self):
        self.var = 3

In [2]:
obj = MyClass()

In [3]:
obj.var

3

In [4]:
MyClass.var

4

In [15]:
from collections import namedtuple
#Trick = namedtuple('Trick', ['__set__', '__get__'])
# trick = object()
# setattr(trick, '__set__', 5)

class TempClass:
    pass

trick = TempClass()
setattr(trick, '__get__', 5)

class MyClass:
    var = trick
    
    def __init__(self):
        self.var = 3

In [16]:
obj = MyClass()

In [17]:
obj.var

3

In [18]:
MyClass.var

In [61]:
from collections import namedtuple
#Trick = namedtuple('Trick', ['__set__', '__get__'])
# trick = object()
# setattr(trick, '__set__', 5)

class TempClass:
    pass

trick = TempClass()
setattr(trick, '__get__', lambda x: 5)

class MyClass:
    var = trick
    
    def __init__(self):
        self.var = 3

In [62]:
obj = MyClass()

In [63]:
obj.var

3

In [64]:
MyClass.var

In [65]:
trick.__get__(3)

5

In [94]:
class TempClass:
    def __get__(self, instance, owner):
        return 5

class MyClass:
    var = TempClass()
    
    def __init__(self):
        self.var = 4

In [95]:
obj = MyClass()

In [96]:
obj.var

4

In [97]:
MyClass.var

5

In [99]:
class TempClass:
    def __get__(self, instance, owner):
        return 5
    
    def __set__(self, instance, value):
        return value

class MyClass:
    var = TempClass()
    
    def __init__(self):
        self.var = 4

In [100]:
obj = MyClass()

In [101]:
obj.var

5

In [102]:
MyClass.var

5

# Line Item Take #4: Automatic Storage Attribute Names

* soln: include class attribute `__counter` in descriptor. Use it to generate unique storage name for each descriptor instance.
    - increment class counter every time a new descriptor is created
    - use hash sign in name to guarantee storage_name will not clash with attributes created by user using dot notation (not valid syntax)
* in this case, you need to implement a get, because the name of the managed attribute is not the same as the storage_name.
    - syntax: `__get(self, instance, owner)__` - the owner argument is a reference to the managed class. Handy when retrieving managed attribute via the class - then get method receives None as the value for instance argument.
* also can now use setattr and getattr on instance because name of storage attribute different than descriptor name - so will not trigger descriptor (thus avoiding infinite recursion)
* 

* Usualy we do not define a descriptor in the same module where it is used, but in a separate utility module designed to be accessed across the applications - even many applications if developing a framework.

### Property factory vs descriptor
* can accomplish pretty much same thing
* he prefers descriptors for two reasons:
    - descriptror class can be extended by sublcassing
    - it is more stratigh forward to hold state in class and instance attribtues than in function and closure ones (asi with counter varaible, and stograge_name)
* on the other hand, property factory does not depend in strange object relationships, evidence by descriptor method arguments self, instance, owner.

# Take 5:
* pretty cool desgin: 
    - have a descriptor class called `Autostorage` that deals with the storing and reading only
    - then have a class that inherits from abc.ABC and AutoStorage, called Validated. 
        * this one has an abstrac method validate, that gets used in an overwritting of `__set__ `
    - Then have to instantiations of validate: Quantity and NonBlank, that provide different validations for the descriptors
    

# Overriding vs Nonoverriding Descriptors
* assymetry in handling attributes: reading attribute to instance normally returns attribute defined in instance, but if there is no such attribute in instance, a class attribute will be retrieved. OTOH assigned attribute in instance, normally creates the attribute in instance, without affecting class at all.
* This also affects descriptors, and leads to creating two categories of descriptions *overriding* vs *non-overriding*, depending on whether `__set__` method is defined
    * a descriptor that implements the `__set__` methods is called an **overriding descriptor** (though usually implement both get and set)
        - this will override attempts to set instance attributes
        - properties are also overriding descriptors - if you dont provide a setter, the defauls `__set__` from the property will raise AttributeError, to singal that attribute is read-only. 
        - if only `__set__` implemented, reading descriptor through instance will return descriptor object itself, untill (unless) the namesake instance attribute is created, at which point reading attribute will return the value from the instance.
            * in other words, instance attribute will shadow description, but only when reading.
     * if not implement set, then it is a **nonoverriding descriptor**.
         - setting instance attribute with same name will shadow descriptor, rendering it ineffective for handling that attribute in that specific instance.
         - *methods are implemented as nonoverriding descriptors* !!Finally, this is how properties work!!??
* overriding descriptors are also sometimes called **data descriptors** or **enforced escriptors** and nonoverriding alled **nondata descriptors** or **shadowable descriptors**


<font color='red'> are these descriptors dispatched differently? or its just same dispatch, but different behavior due to different implemented methods?</font>
    - I think dispatched differently: evidence - write one with a get, and have instance property shadow. What obj.attr does depends on whether irrelevant set method is also written/

# overriding descriptors in the Class
* regardless of whether desriptor is overriding or not, it can be overwrittedn by assignment to class. 
    - this is a monkey-patch technique
* This is another asymetry: reading of class attribute can be controlled by descriptors `__get__`, but the writing cannot be handled by descriptor's `__set__`

# Methods Are (nonoverrdiging) Descriptors
* all user defined functions have a `__get__` method. So they act as descriptors when attached to a class.
* Accessing the function via obj.func vs class.func return different things
    - class.func returns the func itself (accessing get via class)
    - obj.func returns a bound method object: a callable that wraps the function and binds the managed instance to the first argument of the function (i.e. currying)
    - these have different types: **function** vs **method**
* `obj.func` is equivalent to `class.func.__get__(obj)`
    - this returns a bound method.
    - this bound method has a `__self__` attribute holding a reference to the instance on which the method was called.
    - it also contains a `__func__` attribute which is a reference to the original function attached to managed class.
    - it also has a `__call__` method which handles the actual invocation. It calls the original function referenced in `__func__`, passing the self attr as the first argument.
* the way how functions are turned into bound methods is a prime example of how descriptors are used as infrastructure in the language

In [35]:
def myfunc(x):
    return 3

In [37]:
myfunc(0)

3

In [41]:
myfunc.__get__(4)

<bound method myfunc of 4>

In [42]:
myfunc

<function __main__.myfunc(x)>

In [104]:
class MyClass():
    def func(self):
        return self
        

In [105]:
obj = MyClass()

In [106]:
obj.func

<bound method MyClass.func of <__main__.MyClass object at 0x0000015CA2006348>>

In [107]:
obj.func.__self__

In [108]:
obj

In [109]:
obj.func.__func__

<function __main__.MyClass.func(self)>

In [111]:
def func2(x):
    return (x,x)

In [112]:
func2

<function __main__.func2(x)>

In [113]:
obj.func = func2

In [114]:
obj.func

<function __main__.func2(x)>

In [115]:
obj.func(3)

(3, 3)

In [122]:
obj.func.__get__(3)()

(3, 3)

In [123]:
MyClass.func

<function __main__.MyClass.func(self)>

In [124]:
obj.func

<function __main__.func2(x)>

In [125]:
del obj.func

In [126]:
obj.func

<bound method MyClass.func of <__main__.MyClass object at 0x0000015CA2006348>>

# Descriptor Usage Tips
* Use property to keep it simple
    * properties are easiest way to create read-only attributes
* read-only descriptors require `__set__`
    * otherwise namesake attribute will shadow the descriptor
    * set method of read-only should raise AttributeError message
* Validation descriptors can work with `__set__` only. Speeds up getting.
* Caching can be done efficiently with `__get__` only
    * non overriding descriptors useful to make some expensive computation and then cache resutl by setting an instance attribute to shadow descriptor. Subsequent access will get from `__dict__` and not trigger `__get__` anymore.
* Nonspecial methods can be shadowed by instance attributes
    * special methods cant be shadowed, because the interpreter only looks for special methods in the class itself. That is repr(x) is executed as `x.__class__.__repr__(x)`. So a repr attribute in x has no effect on it.

# Other
* docstring of a descriptor class is used to document every instance of the descriptor in the managed class.
    - this is somewhat unsatisfactory because same descriptor might be used for different managed attributes (as opposed to properties, where each one gets its own setter and getter)